## Benchmarking a single CIR process chain

by Antoni Sieminski

### Executive summary

This notebook benchmarks a single run of the CIR process using a CPU. We examine the following settings:

1) No compilation.
    - Timing should be similar to base R.
2) Ahead-of-time (AOT) compilation.
    - This setting compiles the code to run on different machines with a wide variety of inputs.
    - Assuming good software engineering, the performance here should be similar to that of Hymans Robertson's C# code.
3) Just-in-time (JIT) compilation.
    - First run takes longer as it compiles the code to optimise w.r.t. to the specific hardware it's run on and to the supplied function inputs.

Results:
- Some compilation is necessary, but JIT was only twice faster than AOT-compiled code.
- Combined with the additional compilation time of JIT, it might not be the best approch for this problem.

Limitations:
- The biggest parallelisation gains stem from having a large number of chains, which can be vectorised (or rather tensorised) at each iteration.
- This analysis looked at processing only a single chain on a CPU.
- It could be relevant if the chains were run on a cluster, but GPU computing seems like a much easier way to parallelise the computation.

In [3]:
import numpy as np

#parameters
nYears = 100
dt= 1/12
nFactors=2
nTrials = 1
rho = 0.739 
weinerSigma = np.array([[1, rho],
                        [rho, 1]])
weinerL = np.linalg.cholesky(weinerSigma)
weinerL

array([[1.        , 0.        ],
       [0.739     , 0.67370543]])

In [4]:
#initial_values
x_10 = 0.0228
x_20 = 0.0809

#more parameters (alpha etc)
a_1 = 1.0682
m_1 = 0.0546
s_1 = 0.0412
a_2 = 0.0469
m_2 = 0.0778
s_2 = 0.0287

### 1) No compilation

In [5]:



def correlate_weiner_process(X, L, dt):
    return np.sqrt(dt) * np.dot(L, X)

def compute_dxt(xt, dt, alpha, mu, sigma, dZt):
    """
    Parameters:
    - xt: Current value of the process
    - dt: Time increment
    - alpha: Speed of reversion
    - mu: Long-term mean of the process
    - sigma: Volatility parameter
    - dZt: Random increment from a normal distribution

    Returns:
    - Updated process value
    """
    dxt_drift = alpha * (mu - xt) * dt
    dxt_stochastic_variation = sigma * np.sqrt(xt) * dZt  
    return dxt_drift + dxt_stochastic_variation

def generate_process_cir_py(x0, dt, alpha, mu, sigma, dZ):
    """
    Generate a trajectory of the CIR process.

    Parameters:
    - x0: Initial value of the process
    - dt: Time increment
    - alpha: Speed of reversion
    - mu: Long-term mean of the process
    - sigma: Volatility parameter
    - dZ: Array of random increments from a normal distribution

    Returns:
    - Array of process values
    """
    xt = x0
    for dZt in dZ:
        xt += compute_dxt(xt, dt, alpha, mu, sigma, dZt)
    return xt

In [6]:
#random variable X, (2,nfactors*nYears/dt) matrix 
X = np.random.normal(size=int(nFactors * nYears / dt)).astype(np.float32).reshape(2, -1)
#find dZ
dZ = correlate_weiner_process(X,weinerL,dt).astype(np.float32)[0,:]

In [7]:
%timeit x_1 = generate_process_cir_py(x_10, dt, a_1, m_1, s_1, dZ)

2.29 ms ± 166 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### 2) Ahead-of-time compilation (Cython)

In [8]:
%load_ext cython

In [10]:
%%cython
from libc.math cimport sqrt
import numpy as np
cimport numpy as np

cdef inline float updateProcessValue_cir(float xt, float dt, float alpha, float mu, float sigma, float dZt):
    cdef float dxt_drift = alpha * (mu - xt) * dt
    cdef float dxt_stochastic_variation = sigma * sqrt(xt) * dZt
    return xt + dxt_drift + dxt_stochastic_variation

# Improved performance by using memoryviews and reducing Python calls
def generateProcess_cir(float x0, float dt, float alpha, float mu, float sigma, float[:] dZ):
    cdef int T = dZ.shape[0]
    cdef float[:] x = np.empty(T + 1, dtype=np.float32)
    cdef int t
    cdef float xt = x0

    x[0] = x0
    for t in range(1, T + 1): 
        xt = updateProcessValue_cir(xt, dt, alpha, mu, sigma, dZ[t-1])
        x[t] = xt
    return x


In [11]:
# Call the Cython function
%timeit result = generateProcess_cir(x_10, dt, a_1, m_1, s_1, dZ)


17.9 µs ± 293 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Just-in-time compilation (Numba)

In [15]:
import numba
compute_dxt = numba.jit(compute_dxt, nopython=True) # We overwrite the old function to make the code clearer
generate_process_cir_numba = numba.jit(generate_process_cir_py, nopython=True)

In [16]:
# Force compilation
result = generate_process_cir_numba(x_10, dt, a_1, m_1, s_1, dZ)
# Timing after compilation
%timeit result = generate_process_cir_numba(x_10, dt, a_1, m_1, s_1, dZ)

10.2 µs ± 672 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Appendix

In [42]:
# I tried to scale up the cython funciton to multiple dimensions
# %%cython
# from libc.math cimport sqrt
# from cython.parallel import prange
# import numpy as np
# cimport numpy as np

# cdef inline float updateProcessValue_cir(float xt, float dt, float alpha, float mu, float sigma, float dZt):
#     cdef float dxt_drift = alpha * (mu - xt) * dt
#     cdef float dxt_stochastic_variation = sigma * sqrt(xt) * dZt
#     return xt + dxt_drift + dxt_stochastic_variation

# def generateProcess_cir(float x0, float dt, float alpha, float mu, float sigma, float[:] dZ):
#     cdef int T = dZ.shape[0]
#     cdef float[:] x = np.empty(T + 1, dtype=np.float32)
#     cdef int t
#     cdef float xt = x0

#     x[0] = x0
#     for t in range(1, T + 1):
#         xt = updateProcessValue_cir(xt, dt, alpha, mu, sigma, dZ[t-1])
#         x[t] = xt
#     return np.array(x)  # Convert memory view back to a NumPy array for usability outside Cython

# def func(float x0, float dt, float alpha, float mu, float sigma, float[:,:] dZQ):
#     cdef int num_simulations = dZQ.shape[0]
#     cdef int num_points = dZQ.shape[1]
#     cdef float[:,:] Xt = np.empty((num_simulations, num_points + 1), dtype=np.float32)  # +1 for the initial value
#     cdef Py_ssize_t j, i

#     for j in prange(num_simulations, nogil=True):
#         Xt[j, 0] = x0  # Set initial value for each simulation
#         for i in range(1, num_points + 1):
#             Xt[j, i] = updateProcessValue_cir(Xt[j, i - 1], dt, alpha, mu, sigma, dZQ[j, i - 1])
#     return np.array(Xt)  # Convert memory view to NumPy array
